非常に参考になる解説：
https://mosko.tokyo/post/pytorch-rnn/

# RNN


In [5]:
%%bash
wget https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py

wget: /home/toshinao/anaconda3/envs/base2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-02-09 19:50:12--  https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-02-09 19:50:12 ERROR 404: Not Found.



In [2]:
%%bash
git add theoretical_consideraiton.ipynb

In [2]:
%%bash
git commit -a -m "check point"

[master 452d383] check point
 2 files changed, 139 insertions(+), 11 deletions(-)


$$
\begin{align}
\text{時点$t$における$l$番目の(hidden) layerにおけるnode value}=h^{(l)}_t=f
\left(
    W^{(l)}   
    \begin{bmatrix}
        h^{(l-1)}_t\\
        h^{(l)}_{t-1}
    \end{bmatrix}
\right)
\end{align}
$$


* RNNはoption on optionのように考えると理解しやすい

対応は以下のような感じか？

* ひとつめのオプションに対してクーポンが入力される
    * クーポンはembedded vectorになっている「ワード」なので300次元とかのイメージ 
    * なのでそれを受ける一つ目のオプションも300次元の入力を食えるようになっている
    * 300次元は株価方向のラティスをイメージ？
* ひとつめのオプションバリュー自体も300次元であれば株価イメージでいけるがかならずしも同じ次元である必要はない（よね？）
    * その場合の対応するイメージは？？例えば200次元であれば、オプションンバリューのレイヤーは株価方向を粗視化しているとみなすのがいいか？
        *そうだとするとそれってオプションプライシングでも応用可能かも？


# gated RNN

LSTMやGRUなどの総称

soft callのような選択的な期限前償還によりタイムスリップするイメージ
* ただし、満期依存するクーポンでcall notice periodに入るとクーポンのスケジュールもタイムスリップする
* このメカニズムにより非常に遠い満期直前のクーポンの大小が$t=0$におけるオプションバリューに効いてくる


## GRU

(以下、gateの名前が書籍と上記リンクで異なるが書籍のほうを採用する)

* gateによりショートカットしてくるバリュー$n$
    * 普通のRNNやgated RNNとほぼ同じものだが、
    * 一時点前の同レイヤーからのチャンネルがreset gate $r$によって遮断されることがある
* update gate $z$はgated RNNのゲートと同様に機能
*  gate $z$およびupdate gate $r$は普通のRNNのhidden node valueと同じ形式の演算で決まる
    * 書籍の(2.61)式もそのことを物語っている

### option analogy

In [ ]:
* 早期行使が起こるかどうかがupdate gateによってコントロールされている
    * 早期行使

### 以下、古い読解

上記文献の場合、
* candidate node $n$が一番素朴なRNN(活性化関数を$\tanh$としたもの)と自然に対応が付くノード
    * ただし、RNNで素直に一時点前の同じレイヤーの隠れ層からの伝搬があるのに対して、GRUではreset gate $r$がゼロの時は伝搬を遮断する
    * そのreset gateは普通のRNNスタイル（活性化関数はsigmoid）のノード
         * ただし、一時点前のreset gateからの伝搬があるわけではなく、通常のhidden node $h$からの伝搬
* update gate $z$が1になるときは　candidate nodeの値は捨てられて、一時点前の同じレイヤーがそのままスライド採用される
    * これはGated RNNに由来する方法か
    

RNNの設計のうち、オプション的な解釈がむずかしいかもしれないところ
* ひとつめのオプションの現在価値と二つ目のオプションの1ステップ前（すなわち未来）とを混合してから活性化関数にかける
    * ひとつめのオプション価値に依存するクーポンと思えばよいか

# attention

* This is a penというsequenceに二層の隠れ層を用いて、　300x4（4 words $\rightarrow$ 400 x 4 $\rightarrow$ 300 x 4）の出力を得たとする。
* 最後の400および300次元ベクトルがdecoderの隠れ層の初期値となる
    * encoderとdecoderの各隠れ層の次元は同一であるとする
* 翻訳先のdecoderの1単語目の隠れ層出力をまず求める
* 求められた隠れ層出力（300次元ベクトル）とencoderの300次元ベクトル４つそれぞれとの内積を取る
* 内積をウェイトとしたencoderの300次元ベクトルとのweighted sumを求め、もともとのdecoder隠れ層アウトプット300次元とconcatする（600次元になる？）
* 600次元ベクトルを線形変換(600xoutput空間の語彙数)し活性化関数をかける。出力言語の語彙数を10000とすると10000次元のベクトルが出力される
    * 隠れ層がひとつ増える感じ。ただし、普通のRNNの隠れ層ではdecoderの直前ワードの隠れ層をconcatするところ、この最後の「隠れ層」ではattentionがconcatされている

# seq2seq model

* [Seq2Seq in Keras for Petrol Price Prediction using Italian Open Data](https://medium.com/isiway-tech/seq2seq-in-keras-for-petrol-price-prediction-using-italian-open-data-767cb1016af3)はガソリン価格をseq2seqモデルで取り扱う試み


* [High-Dimensional Time Series Forecasting with Convolutional Neural Networks](https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Intro.ipynb)

https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/

# LiNGAM

サンプル次元が列方向になっていることに注意

$$
\begin{align}
x=Bx+e\qquad 
\begin{cases}
B&\text{下三角行列}\\
e &\text{正規分布でない外生変数}
\end{cases}
\end{align}
$$

であるならば$x$の上から順番に因果が流れる

$$
(I-B)x = e
$$

## ICA


$$
\begin{align}
x = A s
\qquad
\begin{cases}
x &\text{観測値}\\
s &\text{hidden indemendend components}
\end{cases}
\end{align}
$$

### サンプル相関：
$$
C = \frac{1}{N} x x^{\rm T}
$$

### サンプル相関の固有値分解

$$
\begin{align}
C = U^{\rm T} \Lambda U
\qquad \text{where} 
\begin{cases}
U&\text{直行行列}\\
\Lambda&\text{対角行列}
\end{cases}
\end{align}
$$

### 無相関にする回転

$$
\epsilon = V\Lambda^{-1} Ux\qquad \text{where $V$ is orthgonal matrix}
$$

さらに対角行列分の自由度があるが意味がないので省略

上記の$\epsilon$がindependent componentと仮定しているので

$$
x = U^{\rm T}\Lambda V^{\rm T} \epsilon
$$



# 参考URL

ただしASPECは利用申請がちょっと面倒そう。。

http://www.phontron.com/japanese-translation-data.php?lang=ja



ASPECというcorpusが便利そう

その他のcorpusへのリンク集
https://qiita.com/imaharu/items/9f206a8f470897d5458c